In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from cryptocmd import CmcScraper #market cap data
from rich.progress import track

In [2]:
#Import all csv and save them in dict with filename (no extension, no USDT) as key
path = "Crypto_Data_Hourly//"
file_names = [f for f in listdir(path) if isfile(join(path, f))]
file_names.remove("BTTCUSDT.csv") #super tiny & no market cap data

dataframe_dict = {}

for file_name in file_names:
    dataframe_dict[file_name[:-8]] = pd.read_csv(path + file_name)
    dataframe_dict[file_name[:-8]]["Open Time"] = pd.to_datetime(dataframe_dict[file_name[:-8]]["Open Time"])

In [3]:
#getting marketcap data
## fixing some names for coinmarketcap data
coinmarketcap_names = {}
for crypto in dataframe_dict:
    coinmarketcap_names[crypto] = crypto
coinmarketcap_names["IOTA"] = "MIOTA" #actually only iota had problems

##scraping coinmarketcap and joining with original dfs
###*FYI can be executed faster if date range is specified in line 12 but I'm too lazy right now to look up syntax and stuff
for crypto in track(dataframe_dict):
    scraper = CmcScraper(coinmarketcap_names[crypto])
    market_cap = scraper.get_dataframe()[["Date", "Market Cap"]]
    market_cap = market_cap.rename(columns = {"Date": "Open Time", "Market Cap": "Market Cap"})
    dataframe_dict[crypto] = pd.merge(dataframe_dict[crypto], market_cap, on = "Open Time")

Output()

In [4]:
#market cap dataframe
all_market_caps = pd.DataFrame()
all_market_caps["Open Time"] = dataframe_dict["BTC"]["Open Time"] #Bitcoin because it has the largest range of dates

for crypto in (dataframe_dict):
    all_market_caps = pd.merge(all_market_caps, dataframe_dict[crypto][["Open Time", "Market Cap"]], on = "Open Time", how = "outer")
    all_market_caps = all_market_caps.rename(columns={all_market_caps.columns[-1]: crypto})
all_market_caps = all_market_caps.fillna(0)

all_market_caps.head(3)

,Open Time,1INCH,AAVE,ADA,ALGO,AMP,APE,AR,ATOM,AVAX,...,XEM,XLM,XMR,XRP,XTZ,YFI,ZEC,ZEN,ZIL,ZRX
0,2017-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-08-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-08-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#relative market cap dataframe
rel_market_caps = all_market_caps.copy(deep=True)

#transpose dataframe
rel_market_caps = rel_market_caps.transpose()
new_header = rel_market_caps.iloc[0] #grab the first row for the header
rel_market_caps = rel_market_caps[1:] #take the data less the header row
rel_market_caps.columns = new_header #set the header row as the df header

#divide each column by sum of market caps to get relative market cap
rel_market_caps = rel_market_caps / rel_market_caps.sum()
assert sum(rel_market_caps.sum()) == 1800 #assert if sums of all dates are 1 * 1800 dates

#transpose dataframe back
rel_market_caps = rel_market_caps.transpose()
rel_market_caps = rel_market_caps.reset_index(level=0)
assert rel_market_caps.columns.all() == all_market_caps.columns.all() #assert if transposing was successful
assert sum(rel_market_caps.loc[:, rel_market_caps.columns != "Open Time"].sum(axis = 1)) == 1800 #assert if sums of all dates are 1 * 1800 dates

rel_market_caps.tail(3)

#TODO: drop all stable coins from the set. There are at least two right now (TUSD & USDP)

,Open Time,1INCH,AAVE,ADA,ALGO,AMP,APE,AR,ATOM,AVAX,...,XEM,XLM,XMR,XRP,XTZ,YFI,ZEC,ZEN,ZIL,ZRX
1797,2022-07-19,0.000476,0.001538,0.02022,0.002984,0.000462,0.002156,0.000598,0.003489,0.008112,...,0.000544,0.003444,0.003082,0.020845,0.001816,0.000286,0.001112,0.000263,0.000664,0.000326
1798,2022-07-20,0.000448,0.001488,0.019517,0.002821,0.000467,0.002123,0.000545,0.003272,0.007888,...,0.000511,0.003312,0.003242,0.020521,0.001722,0.00028,0.001088,0.000249,0.00063,0.000313
1799,2022-07-21,0.00045,0.001544,0.019606,0.002819,0.000453,0.002286,0.000554,0.003683,0.008258,...,0.000513,0.003315,0.003184,0.020609,0.001757,0.000286,0.00107,0.000249,0.000632,0.000317


In [6]:
#TODO: copy the dataframe_dict (deep copy bc dict is mutable) and replace all the price-related columns like Open, High etc. with relative prices
#-> easy to achieve by dividing all the price columns by the last closing price. That way last closing price is 100%. If closing price of
#day before was 10% lower it will have a price of 90% in the new df

#Why? Because if we use absolute prices to create a basket we have the issue that coins with low number of tokens impact the price way more
#than they should. Example: crypto1 has a price of 1€ and in total 1.000 coins -> market cap of 1000€. crypto 2 has a price of 500€
#and in total 2 coins -> market cap of 1000€. 
# If we aggregate by absolute price we might say that the basket crypto has a price of 50% * 1€ + 50% * 500€ = 250.5€

#If crypto 2 decreases by 1% and crypto1 increases by 10% you could say that the market in total
#moved up (both have equal market caps so average of 10% & -1% is positive). But when you aggregate by absolute price
#again you would get a new basket crypto price of [52.6% * 1.1€ + 47.4% * 495] = 235€! So market supposedly went down!
#Relative prices fix this

In [7]:
#TODO aggregate all cryptos (except stable coins) into a basket crypto. Simple sum for volume, number of trades etc. and weighted sumproduct for price related columns
#market cap can also just be summed up, because the market cap is inherently weighed correctly

basket = dataframe_dict["BTC"].copy(deep=True) #Bitcoin because it has the largest range of dates
basket[["Open", "High", "Low", "Close", "Volume", "Quote Asset Volume", "Number of Trades", "TB Base Volume",\
    "TB Quote Volume", "Ignore", "Market Cap"]] = 0
basket

,Open Time,Open Time.1,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore,Market Cap
0,2017-08-17,2017-08-17 00:00:00.000000000,0,0,0,0,0,2017-08-17 23:59:59.999000064,0,0,0,0,0,0
1,2017-08-18,2017-08-18 00:00:00.000000000,0,0,0,0,0,2017-08-18 23:59:59.999000064,0,0,0,0,0,0
2,2017-08-19,2017-08-19 00:00:00.000000000,0,0,0,0,0,2017-08-19 23:59:59.999000064,0,0,0,0,0,0
3,2017-08-20,2017-08-20 00:00:00.000000000,0,0,0,0,0,2017-08-20 23:59:59.999000064,0,0,0,0,0,0
4,2017-08-21,2017-08-21 00:00:00.000000000,0,0,0,0,0,2017-08-21 23:59:59.999000064,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2022-07-17,2022-07-17 00:00:00.000000000,0,0,0,0,0,2022-07-17 23:59:59.999000064,0,0,0,0,0,0
1796,2022-07-18,2022-07-18 00:00:00.000000000,0,0,0,0,0,2022-07-18 23:59:59.999000064,0,0,0,0,0,0
1797,2022-07-19,2022-07-19 00:00:00.000000000,0,0,0,0,0,2022-07-19 23:59:59.999000064,0,0,0,0,0,0
1798,2022-07-20,2022-07-20 00:00:00.000000000,0,0,0,0,0,2022-07-20 23:59:59.999000064,0,0,0,0,0,0
